In [1]:
import cv2
import numpy as np
import pandas as pd
from glob import glob
from tqdm.auto import tqdm
from sklearn.metrics import *
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
import os
import json
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import math
import collections
import random
import numpy as np
import os
from PIL import Image
import time
import random
from keras import backend as K
import pathlib
from sklearn.neighbors import NearestNeighbors

%matplotlib inline
from IPython.display import clear_output

KeyboardInterrupt: 

In [ ]:
seed = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(seed)

# Загрузка данных

In [ ]:
train_captions = pd.read_csv('data/train.csv')
train_captions.head()

## Загрузка фотографий

In [ ]:
data_dir = pathlib.Path('data/train/groups') # необходимо разархивировать архив
image_paths = list(data_dir.glob('*/*.png'))
image_count = len(image_paths)
print(image_count)

In [ ]:
Image.open(str(image_paths[0]))

In [ ]:
batch_size = 128
img_height = 240
img_width = 240

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(
    mode='horizontal_and_vertical',
    seed=seed
    ),
  tf.keras.layers.RandomRotation(
    factor=(-0.1, 0.1),
    seed=seed,
    fill_mode='constant',
    fill_value=255
    ),
  tf.keras.layers.RandomZoom(
    height_factor=(-0.1, 0.1),
    seed=seed,
    ),
  tf.keras.layers.RandomBrightness(
      factor=(-0.2, 0.2),
      seed=seed),
    tf.keras.layers.RandomContrast(
      factor=(0., 0.2), 
      seed=seed
    )
])

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    augmented_images = data_augmentation(images, training=True)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
n_classes = len(class_names)

In [ ]:
# One-hot / categorical encoding
def input_preprocess(image, label):
    label = tf.one_hot(label, n_classes)
    image = data_augmentation(image, training=True)
    return image, label

In [ ]:
train_ds = train_ds.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
val_ds = val_ds.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)

# Модель-Классификатор

In [ ]:
def build_model(num_classes):
    inputs = tf.keras.layers.Input(shape=(img_height, img_width, 3))
    model = EfficientNetV2B2(include_top=False, input_tensor=inputs, weights="imagenet")
    
    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)

    top_dropout_rate = 0.2
    x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
model = build_model(num_classes=n_classes)
model.summary(
    show_trainable=True,
)

In [ ]:
epochs = 8
model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=2)

In [ ]:
def unfreeze_model(model, n_layers):
    # We unfreeze the top n layers while leaving BatchNorm layers frozen
    for layer in model.layers[-n_layers:]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

In [ ]:
epochs = 5
unfreeze_model(model, 10)
model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=2)

In [ ]:
epochs = 5
unfreeze_model(model, 20)
model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=2)

In [ ]:
epochs = 4
unfreeze_model(model, 30)
model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=2)

In [ ]:
model.save('weights/EfficientNetV2B2_v7.h5')

# Тестирование модели (Поиск схожих картинок)

In [ ]:
base_model = load_model('weights/EfficientNetV2B2_v7.h5')
# Customize the model to return features from fully-connected layer
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
model.trainable = False

model.summary(
    show_trainable=True,
)

In [ ]:
from joblib import dump, load

def extract_features(path, model=model):
    img = cv2.imread(path)
    img = tf.keras.layers.Resizing(240, 240)(img)
    x = np.expand_dims(img, axis=0)

    feature = model.predict(x, verbose=0)[0]

    return feature / np.linalg.norm(feature)

In [ ]:
train_embeddings = []

for idx in tqdm(train_captions.idx):
    train_embeddings.append(extract_features(f'data/train/{idx}.png'))

train_embeddings = np.array(train_embeddings)

queries_embeddings = []

for idx in tqdm(train_captions.idx[1000:1300]):
    queries_embeddings.append(extract_features(f'data/train/{idx}.png'))

queries_embeddings = np.array(queries_embeddings)

In [ ]:
from sklearn.decomposition import PCA, KernelPCA

mapper = PCA(n_components=1000)
mapper.fit(train_embeddings)

exp_var_pca = mapper.explained_variance_ratio_
#
# Cumulative sum of eigenvalues; This will be used to create step plot
# for visualizing the variance explained by each principal component.
#
cum_sum_eigenvalues = np.cumsum(exp_var_pca)
#
# Create the visualization plot
#
plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
mapper = KernelPCA(n_components=400, kernel='rbf')
mapper.fit(train_embeddings)

dump(mapper, 'PCA_400_v7_rbf.joblib')

In [ ]:
mapper = load('PCA_400_v7_rbf.joblib')

In [ ]:
train_embedded = mapper.transform(train_embeddings)
queries_embedded = mapper.transform(queries_embeddings)

In [ ]:
neigh = NearestNeighbors(n_neighbors=16, metric='cosine', algorithm='brute')
neigh.fit(train_embedded)

distances, idxs = neigh.kneighbors(queries_embedded, 16, return_distance=True)

In [ ]:
pred_data = pd.DataFrame()
pred_data['score'] = distances.flatten()
pred_data['database_idx'] = [train_captions.index[x] for x in idxs.flatten()]
pred_data.loc[:, 'query_idx'] = np.repeat(train_captions.index[1000:1300], 16).values

In [ ]:
pred_data.score = pred_data.score.apply(lambda x: 1 - x)
pred_data.score = (pred_data.score - pred_data.score.min(axis=0)) / (pred_data.score.max(axis=0) - pred_data.score.min(axis=0))
pred_data.score.mean()

In [ ]:
import nltk
import pymorphy2
import razdel
from string import punctuation

nltk.download('stopwords') # скачиваем стоп-слова
ru_stopwords = nltk.corpus.stopwords.words('russian')
stop_words = ['...', 'cbn-быть', 'камаз-евро', 'верхний','высокий','г-образный','жёлтый','зелёный',
              'красный','левый','синий','наружный','неподвижный','передний','п-образный','правый', 'т-смарт', 'т-flex',
             'уралец', 'маз', 'камаз', 'медный', 'металлический', 'люкс', 'лобовой', 'коммунальный', 'вс-т', 'влево', 'вправо',
             'задний', 'передний', 'ведущий', 'ведомый', 'болгария', 'греция', 'боковой', 'кабинный', 'азотировать', 'аналог',
             'замедлительный', 'ямз', 'мом', 'торможение', 'прямой', 'простой', 'проходной', 'синтая', 'призматический', 
             'тупой', 'упираться', 'самоустанавливаться', 'радиальный', 'шариковый', 'плунжерный', 'подъёмный', 'трактор',
             'раструбный', 'раструструбный', 'подвижный', 'русич', 'мпа', 'торцевой', 'лодочный', 'четверной', 'универсальный',
             'ход', 'оцинк', 'стопорный', 'круглый', 'сечение', 'квадрат', 'цвет', 'накал', 'нижний', 'верхний', 'солнечный',
             'конический', 'файтереть', 'номерной', 'рубашка', 'сцепление', 'давление', 'поворотный', 'рулевой', 'гладкий', 
             'сталь', 'нержавеющий', 'поворот', 'меш', 'всасывать', 'червячный', 'регулировочный', 'сельскохозяйственный', 
             'пку', 'профильный', 'повышать', 'понижать', 'шаровыя', 'распределительный', 'промежуточный', 'первичный', 
             'бульдозерный', 'ротационный', 'проч', 'пенька', 'ниточный', 'впускной', 'закалённый', 'мотоблок', 'голый', 'клапанный',
             'новый', 'капот', 'штанга', 'цифровой', 'стремянка', 'секция', 'электрический', 'тарелка', 'резьбовой', 'скаут',
             'универсал', 'сменный', 'средний', 'подключение', 'площадка', 'наружний', 'продольный', 'модификация', 'сетка', 
             'стрельчатый', 'прерывание', 'непрерывный', 'ходовой', 'приводной', 'средний', 'реверсивный', 'соединительный',
             'кпс', 'ксп', 'стык', 'отверстый', 'маслосъёмный', 'раздвижной', 'предохранительный', 'тарелка', 'тсн', 'подогрев',
             'хранение', 'урал', 'охлаждать', 'полевой', 'уралец', 'сеновязальный', 'джутовый', 'внешний', 'стальной', 'грубый',
             'система', 'ожлаждение', 'коса', 'напорный', 'дистанционный', 'крестовидный', 'гидравлический', 'дизельный',
             'двойной', 'толщина', 'жечь', 'крестовина', 'имбусовый', 'байонетный', 'навесный', 'усиленный', 'разбрасывать',
             'жсу', 'год', 'мех', 'большой', 'малый', 'резьба', 'переходной', 'бум', 'бумажный', 'маслоналивной', 'плоский',
             'внешний', 'внешн', 'грубый', 'серия', 'распорный', 'стальной', 'бумага', 'подрулевой', 'внутренний', 'раздаточный',
             'роликовый', 'квт', 'градус', 'бобина', 'тонкий', 'зил', 'линейный', 'прозрачный', 'короткий', 'форкамерный']
morph = pymorphy2.MorphAnalyzer()

def preproc(sentence):
    if 'Втулка' in sentence:
        sentence += ' Втулка'
    if 'Пластина' in sentence:
        sentence += ' Пластина'
    tokens = [_.text.lower() for _ in list(razdel.tokenize(str(sentence)))]
    unique_tokens = set()

    final_sentence = []

    for token in tokens:
        if (len(token) > 2 and not any([char.isdigit() for char in token])):
            parsed_token = morph.parse(token)[0]
            if str(parsed_token.tag) != 'LATN' \
            and parsed_token.normal_form not in ru_stopwords and parsed_token.normal_form not in punctuation \
            and parsed_token.normal_form not in stop_words:
                
                token = str(parsed_token.normal_form)
                if token in ['u-болт', 'болт-скоба']:
                    token = 'болт'
                elif token == 'гидробак':
                    token = 'бак'
                elif token == 'полуось':
                    token = 'ось'
                elif token == 'проводы':
                    token = 'провод'
                elif token == 'фильтровать':
                    token = 'фильтр'
                elif 'пресс' in token:
                    token = 'пресс'
                elif 'хладон' in token:
                    token = 'баллон'
                elif 'поршневой' in token:
                    token = 'поршень'
                elif 'гайка' in token:
                    token = 'гайка'
                elif 'фреон' in token:
                    token = 'газ'
                elif 'трактор' in token:
                    token = 'трактор'
                elif 'датчик' in token:
                    token = 'датчик'
                elif 'ремкомплект' in token:
                    token = 'ремень'
                elif 'упак' in token:
                    token = 'упаковка'
                elif 'переходный' in token:
                    token = 'переходник'
                elif 'упл' in token:
                    token = 'уплотнитель'
                elif 'компл' in token:
                    token = 'комплект'
                elif 'стартер' in token:
                    token = 'стартер'
                elif 'шатунно-поршневой' in token:
                    token = 'шатун'
                elif 'форсунка-распылитель' in token:
                    token = 'форсунка'
                elif 'фиттинг' in token:
                    token = 'фитинг'
                elif 'всасывать' in token:
                    token = 'вентилятор'
                elif 'валик' in token:
                    token = 'вал'
                elif 'агретирование' in token:
                    token = 'навеска'
                elif 'агрегатирование' in token:
                    token = 'навеска'
                elif 'гидронасос' in token:
                    token = 'гидравлический насос'
                elif 'шарикоподшипник' in token:
                    token = 'подшипник'
                elif 'ролик-натяжитель' in token:
                    token = 'ролик натяжитель'
                    
                if token not in unique_tokens:
                    final_sentence.append(token)
                    unique_tokens.add(token)
    if not final_sentence:
        final_sentence = ['другое']
    return final_sentence

In [ ]:
tqdm.pandas() # чтобы отображалось время применения функции

%time train_captions['item_nm'] = train_captions['item_nm'].progress_apply(preproc)
train_captions['item_nm'] = train_captions['item_nm'].apply(lambda x: x[0])

In [ ]:
submit = pred_data.groupby(by='query_idx').apply(lambda dft: dft.nlargest(1, 'score')) \
.merge(train_captions, left_on='database_idx', right_on='idx')
submit.drop(['score', 'idx', 'database_idx'], inplace=True, axis=1)

In [ ]:
submit = pred_data.merge(submit, how='left', on='query_idx')
submit = submit.rename({'item_nm': 'key'}, axis=1)
submit = submit.merge(train_captions, left_on='database_idx', right_on='idx').drop(['idx'], axis=1)

In [ ]:
submit['marker'] = submit.apply(lambda x: str(x.key) in str(x.item_nm), axis=1)
submit['score_2'] = submit.apply(lambda x: (x.score + x.marker) / 2 if x.marker else x.score / 2, axis=1)
submit.drop(['item_nm', 'key', 'marker', 'score'], inplace=True, axis=1)
submit = submit.sort_values(['query_idx', 'score_2'], ascending=[True, False]).reset_index(drop=True)
submit = submit.rename({'score_2': 'score'}, axis=1)
submit

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(cv2.imread(f'data/train/1299.png').astype("uint8"))

In [ ]:
plt.figure(figsize=(10, 10))
image_idx = submit[submit.query_idx == 1299].database_idx.values

for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(cv2.imread(f'data/train/{image_idx[i]}.png'))
    plt.axis("off")